In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Introducción a las GANs
==============


Redes Generativas Adversarias
============================

¿Qué es una GAN?
---------------

Las GANs son un marco para enseñar a un modelo de aprendizaje profundo a capturar 
la distribución de los datos de entrenamiento para poder generar nuevos datos de esa 
misma distribución. Las GANs fueron inventadas por Ian Goodfellow en 2014 y descritas 
por primera vez en el artículo [Generative Adversarial Nets](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf). 
Están formadas por dos modelos distintos, un *generador* y un *discriminador*. El 
trabajo del generador es crear imágenes 'falsas' que parezcan imágenes de entrenamiento. 
El trabajo del discriminador es mirar una imagen y determinar si es una imagen real 
de entrenamiento o una imagen falsa del generador. Durante el entrenamiento, el 
generador está constantemente tratando de engañar al discriminador generando mejores 
falsificaciones, mientras que el discriminador está trabajando para convertirse en 
un mejor detective y clasificar correctamente las imágenes reales y falsas. El 
equilibrio de este juego se alcanza cuando el generador está generando falsificaciones 
perfectas que parecen provenir directamente de los datos de entrenamiento, y el 
discriminador se queda con una confianza del 50% al adivinar si la salida del 
generador es real o falsa.

Ahora, definamos algunas notaciones que se utilizarán a lo largo del tutorial, 
comenzando con el discriminador. Sea $x$ los datos que representan una imagen. 
$D(x)$ es la red discriminadora que produce la probabilidad (escalar) de que $x$ 
provenga de los datos de entrenamiento en lugar del generador. Aquí, como estamos 
tratando con imágenes, la entrada a $D(x)$ es una imagen de tamaño CHW 3x64x64. 
Intuitivamente, $D(x)$ debería ser ALTO cuando $x$ proviene de los datos de 
entrenamiento y BAJO cuando $x$ proviene del generador. $D(x)$ también puede 
considerarse como un clasificador binario tradicional.

Para la notación del generador, sea $z$ un vector de espacio latente muestreado 
de una distribución normal estándar. $G(z)$ representa la función generadora que 
mapea el vector latente $z$ al espacio de datos. El objetivo de $G$ es estimar 
la distribución de la que provienen los datos de entrenamiento ($p_{data}$) para 
poder generar muestras falsas de esa distribución estimada ($p_g$).

Así, $D(G(z))$ es la probabilidad (escalar) de que la salida del generador $G$ 
sea una imagen real. Como se describe en el [artículo de Goodfellow](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf), 
$D$ y $G$ juegan un juego minimax en el que $D$ intenta maximizar la probabilidad 
de clasificar correctamente reales y falsos ($logD(x)$), y $G$ intenta minimizar 
la probabilidad de que $D$ prediga que sus salidas son falsas ($log(1-D(G(z)))$). 
Del artículo, la función de pérdida GAN es

$$\underset{G}{\text{min}} \underset{D}{\text{max}}V(D,G) = \mathbb{E}_{x\sim p_{data}(x)}\big[logD(x)\big] + \mathbb{E}_{z\sim p_{z}(z)}\big[log(1-D(G(z)))\big]$$

En teoría, la solución a este juego minimax es donde $p_g = p_{data}$, y el 
discriminador adivina aleatoriamente si las entradas son reales o falsas. Sin 
embargo, la teoría de convergencia de las GANs todavía está siendo investigada 
activamente y en realidad los modelos no siempre se entrenan hasta este punto.

¿Qué es una DCGAN?
-----------------

Una DCGAN es una extensión directa de la GAN descrita anteriormente, excepto que 
utiliza explícitamente capas convolucionales y convolucionales-transpuestas en el 
discriminador y generador, respectivamente. Fue descrita por primera vez por 
Radford et. al. en el artículo [Unsupervised Representation Learning With Deep 
Convolutional Generative Adversarial Networks](https://arxiv.org/pdf/1511.06434.pdf). 
El discriminador está compuesto por capas de [convolución](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d) 
con stride, capas de [batch norm](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm2d) 
y activaciones [LeakyReLU](https://pytorch.org/docs/stable/nn.html#torch.nn.LeakyReLU). 
La entrada es una imagen de 3x64x64 y la salida es una probabilidad escalar de que 
la entrada provenga de la distribución de datos reales. El generador está compuesto 
por capas [convolucionales-transpuestas](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d), 
capas de batch norm y activaciones [ReLU](https://pytorch.org/docs/stable/nn.html#relu). 
La entrada es un vector latente, $z$, que se extrae de una distribución normal 
estándar y la salida es una imagen RGB de 3x64x64. Las capas conv-transpuestas 
con stride permiten que el vector latente se transforme en un volumen con la misma 
forma que una imagen. En el artículo, los autores también dan algunos consejos 
sobre cómo configurar los optimizadores, cómo calcular las funciones de pérdida 
y cómo inicializar los pesos del modelo, todo lo cual se explicará en las 
secciones siguientes.


In [1]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

Random Seed:  999


Entradas
========

Definamos algunos términos para la ejecución:

- `dataroot` - la ruta al directorio raíz del conjunto de datos. Hablaremos más sobre el conjunto de datos en la siguiente sección.
- `workers` - el número de hilos trabajadores para cargar los datos con el `DataLoader`.
- `batch_size` - el tamaño del lote usado en el entrenamiento. Nuestra GAN usa un tamaño de lote de 128.
- `image_size` - el tamaño espacial de las imágenes usadas para el entrenamiento. Esta implementación usa por defecto 64x64. Si se desea otro tamaño, las estructuras de D y G deben ser modificadas.
- `nc` - número de canales de color en las imágenes de entrada. Para imágenes a color esto es 3.
- `nz` - longitud del vector latente.
- `ngf` - relacionado con la profundidad de los mapas de características propagados por el generador.
- `ndf` - establece la profundidad de los mapas de características propagados por el discriminador.
- `num_epochs` - número de épocas de entrenamiento a ejecutar. Entrenar por más tiempo probablemente lleve a mejores resultados pero también tomará mucho más tiempo.
- `lr` - tasa de aprendizaje para el entrenamiento. Como se describe en el artículo DCGAN, este número debe ser 0.0002.
- `beta1` - hiperparámetro beta1 para los optimizadores Adam. Como se describe en el artículo, este número debe ser 0.5.
- `ngpu` - número de GPUs disponibles. Si es 0, el código se ejecutará en modo CPU. Si este número es mayor que 0 se ejecutará en esa cantidad de GPUs.


In [2]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

Datos
=====

En este tutorial usaremos el [conjunto de datos de rostros Celeb-A](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) que se puede descargar en el sitio enlazado o en [One Drive](). El conjunto de datos se descargará como un archivo llamado `img_align_celeba.zip`. Una vez descargado, cree un directorio llamado `celeba` y extraiga el archivo zip en ese directorio. Luego, establezca la entrada `dataroot` para este notebook en el directorio `celeba` que acaba de crear. La estructura de directorios resultante debería ser:



In [ ]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

Implementación
==============

Con nuestros parámetros de entrada configurados y el conjunto de datos preparado, 
ahora podemos profundizar en la implementación. Comenzaremos con la estrategia 
de inicialización de pesos, luego hablaremos en detalle sobre el generador, el 
discriminador, las funciones de pérdida y el bucle de entrenamiento.

Inicialización de Pesos
-----------------------

Según el artículo DCGAN, los autores especifican que todos los pesos del modelo 
deben inicializarse aleatoriamente a partir de una distribución Normal con 
`media=0` y `desviación_estándar=0.02`. La función `weights_init` toma un modelo 
inicializado como entrada y reinicializa todas las capas convolucionales, 
convolucionales-transpuestas y de normalización por lotes para cumplir con estos 
criterios. Esta función se aplica a los modelos inmediatamente después de su 
inicialización.


In [ ]:
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

Generador
=========

El generador, $G$, está diseñado para mapear el vector del espacio latente ($z$) al espacio de datos. Como nuestros datos son imágenes, convertir $z$ al espacio de datos significa crear una imagen RGB del mismo tamaño que las imágenes de entrenamiento (es decir, 3x64x64). En la práctica, esto se logra a través de una serie de capas convolucionales transpuestas bidimensionales con stride, cada una emparejada con una capa de normalización por lotes 2d y una activación relu. La salida del generador pasa por una función tanh para devolverla al rango de datos de entrada de $[-1,1]$. Vale la pena notar la existencia de las funciones de normalización por lotes después de las capas conv-transpuestas, ya que esta es una contribución crítica del artículo DCGAN. Estas capas ayudan con el flujo de gradientes durante el entrenamiento. A continuación se muestra una imagen del generador del artículo DCGAN.

![](https://pytorch.org/tutorials/_static/img/dcgan_generator.png)

Observa cómo las entradas que establecimos en la sección de entrada (`nz`, `ngf` y `nc`) influyen en la arquitectura del generador en el código. `nz` es la longitud del vector de entrada z, `ngf` se relaciona con el tamaño de los mapas de características que se propagan a través del generador, y `nc` es el número de canales en la imagen de salida (establecido en 3 para imágenes RGB). A continuación está el código del generador.


In [ ]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

Ahora podemos crear una instancia del generador y aplicar la función `weights_init`. 
Observa el modelo impreso para ver cómo está estructurado el objeto generador.


In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
print(netG)

Discriminador
=============

Como se mencionó, el discriminador, $D$, es una red de clasificación binaria que toma una imagen como entrada y produce una probabilidad escalar de que la imagen de entrada sea real (en contraposición a falsa). Aquí, $D$ toma una imagen de entrada de 3x64x64, la procesa a través de una serie de capas Conv2d, BatchNorm2d y LeakyReLU, y produce la probabilidad final a través de una función de activación Sigmoid. Esta arquitectura se puede extender con más capas si es necesario para el problema, pero hay aspectos significativos en el uso de la convolución con stride, BatchNorm y LeakyReLUs. El artículo DCGAN menciona que es una buena práctica usar convolución con stride en lugar de pooling para reducir el muestreo porque permite que la red aprenda su propia función de pooling. Además, las funciones de normalización por lotes y leaky relu promueven un flujo saludable del gradiente, lo cual es crítico para el proceso de aprendizaje tanto de $G$ como de $D$.


Código del discriminador


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is ``(nc) x 64 x 64``
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf) x 32 x 32``
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*2) x 16 x 16``
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*4) x 8 x 8``
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(ndf*8) x 4 x 4``
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

Ahora, al igual que con el generador, podemos crear el discriminador, aplicar la
función `weights_init` e imprimir la estructura del modelo.


In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    
# Apply the ``weights_init`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

# Print the model
print(netD)

Funciones de Pérdida y Optimizadores
===================================

Con $D$ y $G$ configurados, podemos especificar cómo aprenden a través de las funciones de pérdida y los optimizadores. Usaremos la función de pérdida de Entropía Cruzada Binaria ([BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html#torch.nn.BCELoss)) que está definida en PyTorch como:

$$\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad l_n = - \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right]$$

Note cómo esta función proporciona el cálculo de ambos componentes logarítmicos en la función objetivo (es decir, $log(D(x))$ y $log(1-D(G(z)))$). Podemos especificar qué parte de la ecuación BCE usar con la entrada $y$. Esto se logra en el bucle de entrenamiento que viene pronto, pero es importante entender cómo podemos elegir qué componente deseamos calcular simplemente cambiando $y$ (es decir, las etiquetas GT).

A continuación, definimos nuestra etiqueta real como 1 y la etiqueta falsa como 0. Estas etiquetas se usarán al calcular las pérdidas de $D$ y $G$, y esta es también la convención utilizada en el artículo GAN original. Finalmente, configuramos dos optimizadores separados, uno para $D$ y otro para $G$. Como se especifica en el artículo DCGAN, ambos son optimizadores Adam con tasa de aprendizaje 0.0002 y Beta1 = 0.5. Para hacer seguimiento del progreso de aprendizaje del generador, generaremos un lote fijo de vectores latentes extraídos de una distribución Gaussiana (es decir, fixed_noise). En el bucle de entrenamiento, periódicamente introduciremos este fixed_noise en $G$, y a lo largo de las iteraciones veremos cómo se forman imágenes a partir del ruido.


In [ ]:
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

Entrenamiento
============

Finalmente, ahora que tenemos definidas todas las partes del marco GAN, podemos 
entrenarla. Ten en cuenta que entrenar GANs es algo así como un arte, ya que una 
configuración incorrecta de hiperparámetros lleva al colapso del modo sin mucha 
explicación de lo que salió mal. Aquí, seguiremos de cerca el Algoritmo 1 del 
[artículo de Goodfellow](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf), 
mientras seguimos algunas de las mejores prácticas mostradas en 
[ganhacks](https://github.com/soumith/ganhacks). Específicamente, "construiremos 
diferentes mini-lotes para imágenes reales y falsas", y también ajustaremos la 
función objetivo de G para maximizar $log(D(G(z)))$. El entrenamiento se divide 
en dos partes principales. La Parte 1 actualiza el Discriminador y la Parte 2 
actualiza el Generador.

**Parte 1 - Entrenar el Discriminador**

Recuerda, el objetivo de entrenar el discriminador es maximizar la probabilidad 
de clasificar correctamente una entrada dada como real o falsa. En términos de 
Goodfellow, deseamos "actualizar el discriminador ascendiendo su gradiente 
estocástico". En la práctica, queremos maximizar $log(D(x)) + log(1-D(G(z)))$. 
Debido a la sugerencia de mini-lotes separados de [ganhacks](https://github.com/soumith/ganhacks), 
calcularemos esto en dos pasos. Primero, construiremos un lote de muestras reales 
del conjunto de entrenamiento, haremos un pase hacia adelante a través de $D$, 
calcularemos la pérdida ($log(D(x))$), y luego calcularemos los gradientes en un 
pase hacia atrás. En segundo lugar, construiremos un lote de muestras falsas con 
el generador actual, haremos un pase hacia adelante de este lote a través de $D$, 
calcularemos la pérdida ($log(1-D(G(z)))$), y *acumularemos* los gradientes con 
un pase hacia atrás. Ahora, con los gradientes acumulados tanto de los lotes 
completamente reales como completamente falsos, llamamos a un paso del optimizador 
del Discriminador.

**Parte 2 - Entrenar el Generador**

Como se indica en el artículo original, queremos entrenar el Generador minimizando 
$log(1-D(G(z)))$ en un esfuerzo por generar mejores falsificaciones. Como se 
mencionó, Goodfellow demostró que esto no proporciona gradientes suficientes, 
especialmente al principio del proceso de aprendizaje. Como solución, en su lugar 
deseamos maximizar $log(D(G(z)))$. En el código logramos esto: clasificando la 
salida del Generador de la Parte 1 con el Discriminador, calculando la pérdida de 
G *usando etiquetas reales como GT*, calculando los gradientes de G en un pase 
hacia atrás, y finalmente actualizando los parámetros de G con un paso del 
optimizador. Puede parecer contraintuitivo usar las etiquetas reales como etiquetas 
GT para la función de pérdida, pero esto nos permite usar la parte $log(x)$ de la 
`BCELoss` (en lugar de la parte $log(1-x)$) que es exactamente lo que queremos.

Finalmente, haremos algunos informes estadísticos y al final de cada época 
pasaremos nuestro lote fixed_noise a través del generador para rastrear 
visualmente el progreso del entrenamiento de G. Las estadísticas de entrenamiento 
reportadas son:

- **Loss_D** - pérdida del discriminador calculada como la suma de pérdidas para 
    los lotes completamente reales y completamente falsos ($log(D(x)) + log(1 - D(G(z)))$).
- **Loss_G** - pérdida del generador calculada como $log(D(G(z)))$
- **D(x)** - la salida promedio (a través del lote) del discriminador para el lote 
    completamente real. Esto debería comenzar cerca de 1 y luego teóricamente 
    converger a 0.5 cuando G mejore. Piensa por qué es esto.
- **D(G(z))** - salidas promedio del discriminador para el lote completamente falso. 
    El primer número es antes de que D se actualice y el segundo número es después 
    de que D se actualice. Estos números deberían comenzar cerca de 0 y converger a 
    0.5 a medida que G mejore. Piensa por qué es esto.

**Nota:** Este paso puede llevar un tiempo, dependiendo de cuántas épocas ejecutes 
y si eliminaste algunos datos del conjunto de datos.


In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1

Resultados
==========

Finalmente, veamos cómo nos fue. Aquí, examinaremos tres resultados diferentes. 
Primero, veremos cómo cambiaron las pérdidas de D y G durante el entrenamiento. 
Segundo, visualizaremos la salida de G en el lote fixed_noise para cada época. Y 
tercero, veremos un lote de datos reales junto a un lote de datos falsos de G.

**Pérdida versus iteración de entrenamiento**

A continuación se muestra una gráfica de las pérdidas de D y G versus las 
iteraciones de entrenamiento.


In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

**Visualización del progreso del Generador**

Recuerda que guardamos la salida del generador en el lote fixed_noise después de cada época de entrenamiento. Ahora, podemos visualizar la progresión del entrenamiento de G con una animación. Presiona el botón de reproducción para iniciar la animación.


In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

**Imágenes Reales vs. Imágenes Falsas**

Finalmente, veamos algunas imágenes reales y falsas lado a lado.


In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()